<a href="https://colab.research.google.com/github/AweSamarth/boilerplate-book-recommendation-engine/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [74]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [75]:
# add your code here - consider creating a new cell for each section of code
df_books=(df_books[(df_books['author']!="") & (df_books['isbn']!="") & (df_books['title']!="")])
# print(df_ratings)
df_ratings_userrem=df_ratings[df_ratings['user'].isin(df_ratings['user'].value_counts()[df_ratings['user'].value_counts()<200].index)]
df_ratings_bookrem=df_ratings[df_ratings['isbn'].isin(df_ratings['isbn'].value_counts()[df_ratings['isbn'].value_counts()<100].index)]
# print(df_ratings_bookrem)
# print(df_ratings_userrem)

df_ratings=df_ratings[~df_ratings['user'].isin(df_ratings_userrem['user'])]
df_ratings=df_ratings[~df_ratings['isbn'].isin(df_ratings_bookrem['isbn'])]
print(df_ratings)




           user        isbn  rating
1456     277427  002542730X    10.0
1469     277427  0060930535     0.0
1471     277427  0060934417     0.0
1474     277427  0061009059     9.0
1484     277427  0140067477     0.0
...         ...         ...     ...
1147304  275970  0804111359     0.0
1147436  275970  140003065X     0.0
1147439  275970  1400031346     0.0
1147440  275970  1400031354     0.0
1147441  275970  1400031362     0.0

[49781 rows x 3 columns]


In [76]:
df_merged= pd.merge(df_books, df_ratings, on='isbn')
print(df_merged)
# print(df_merged.shape)
# df_merged=df_merged[df_merged['user'].isin(df_merged['user'].value_counts()[df_merged['user'].value_counts()>199].index)]
# df_merged=df_merged[df_merged['isbn'].isin(df_merged['isbn'].value_counts()[df_merged['isbn'].value_counts()>99].index)]
# print(df_merged)


             isbn                                              title  \
0      0440234743                                      The Testament   
1      0440234743                                      The Testament   
2      0440234743                                      The Testament   
3      0440234743                                      The Testament   
4      0440234743                                      The Testament   
...           ...                                                ...   
49512  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   
49513  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   
49514  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   
49515  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   
49516  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   

                  author    user  rating  
0           John Grisham  277478     0.0  
1           John Grisham    2977     0.0  
2     

In [82]:
# matrix=csr_matrix((df_merged['isbn'], df_merged['rating'].values), shape=(2,2))
# print(matrix)
rating_books_pivoted=df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0.0)
print(rating_books_pivoted)

user                                                254     2276    2766    \
title                                                                        
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
A Beautiful Mind: The Life of Mathematical Geni...     0.0     0.0     0.0   
...                                                    ...     ...     ...   
Without Remorse                                        0.0     0.0     0.0   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0

In [85]:
model=NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=7)
model.fit(rating_books_pivoted)

NearestNeighbors(metric='cosine', n_neighbors=7)

In [ ]:
the_books=model.kneighbors(rating_books_pivoted.loc[["Where the Heart Is (Oprah's Book Club (Paperback))"]], 5, return_distance=True)
print(the_books)
temparray=np.array(the_books[1][0])
print(temparray)
anothertemparray= np.array(the_books[0][0])
print(anothertemparray)
# print(rating_books_pivoted.index[the_books[1]][0])
for value in (rating_books_pivoted.index[temparray]):
    print(value)


In [119]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # print(book)
  req_array=[]
  req_array.append([book])
  req_array.append([])
  # print(req_array)
  the_books=model.kneighbors(rating_books_pivoted.loc[[book]], 5, return_distance=True)
  temparray=np.array(the_books[1][0])
  anothertemparray= np.array(the_books[0][0])
  # print(rating_books_pivoted.index[temparray])
  for i in range(len(the_books[1][0])-1, 0, -1):
    print(rating_books_pivoted.index[the_books[1][0][i]])

  # print(the_books)
  # return recommended_books

get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

The Weight of Water
The Surgeon
I Know This Much Is True
The Lovely Bones: A Novel


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()